<a href="https://colab.research.google.com/github/luoChengwen/Agentic-AI/blob/main/LLM_Function_Calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!!pip install litellm

# Important!!!
#
# <---- Set your 'OPENAI_API_KEY' as a secret over there with the "key" icon
#
#
import os
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = api_key

In [ ]:
import json
import os
from typing import List

from litellm import completion

def list_files() -> List[str]:
    """List files in the current directory."""
    return os.listdir(".")

def read_file(file_name: str) -> str:
    """Read a file's contents."""
    try:
        with open(file_name, "r") as file:
            return file.read()
    except FileNotFoundError:
        return f"Error: {file_name} not found."
    except Exception as e:
        return f"Error: {str(e)}"


tool_functions = {
    "list_files": list_files,
    "read_file": read_file
}

tools = [
    {
        "type": "function",
        "function": {
            "name": "list_files",
            "description": "Returns a list of files in the directory.",
            "parameters": {"type": "object", "properties": {}, "required": []}
        }
    },
    {
        "type": "function",
        "function": {
            "name": "read_file",
            "description": "Reads the content of a specified file in the directory.",
            "parameters": {
                "type": "object",
                "properties": {"file_name": {"type": "string"}},
                "required": ["file_name"]
            }
        }
    }
]

# Our rules are simplified since we don't have to worry about getting a specific output format
agent_rules = [{
    "role": "system",
    "content": """
You are an AI agent that can perform tasks by using available tools.

If a user asks about files, documents, or content, first list the files before reading them.
"""
}]

user_task = input("What would you like me to do? ")

memory = [{"role": "user", "content": user_task}]

messages = agent_rules + memory

response = completion(
    model="openai/gpt-4o",
    messages=messages,
    tools=tools,
    max_tokens=1024
)

# Extract the tool call from the response, note we don't have to parse now!
tool = response.choices[0].message.tool_calls[0]
tool_name = tool.function.name
tool_args = json.loads(tool.function.arguments)
result = tool_functions[tool_name](**tool_args)

print(f"Tool Name: {tool_name}")
print(f"Tool Arguments: {tool_args}")
print(f"Result: {result}")

What would you like me to do? what is the file 'README.md' about
Tool Name: list_files
Tool Arguments: {}
Result: ['.config', 'README.md', 'sample_data']


In [ ]:
response.choices[0]

Choices(finish_reason='tool_calls', index=0, message=Message(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(function=Function(arguments='{}', name='list_files'), id='call_7lz1mTnwlJk4psR3gDC5pWuL', type='function')], function_call=None, provider_specific_fields={'refusal': None}, annotations=[]), provider_specific_fields={})

In [ ]:
tool

ChatCompletionMessageToolCall(function=Function(arguments='{}', name='list_files'), id='call_7lz1mTnwlJk4psR3gDC5pWuL', type='function')

In [ ]:


user_task = input("What would you like me to do? ")

memory = [{"role": "user", "content": user_task}]

messages = agent_rules + memory



MAX_STEPS = 5
for step in range(MAX_STEPS):
    response = completion(
        model="openai/gpt-4o",
        messages=messages,
        tools=tools,
        max_tokens=1024
    )

    message = response.choices[0].message
    print(message)
    # Extract the tool call from the response, note we don't have to parse now!
    tool = response.choices[0].message.tool_calls[0]
    tool_name = tool.function.name
    tool_args = json.loads(tool.function.arguments)
    result = tool_functions[tool_name](**tool_args)

    print(f"Tool Name: {tool_name}")
    print(f"Tool Arguments: {tool_args}")
    print(f"Result: {result}")
    messages.append({"role": "assistant", "tool_calls": [tool]})
    print('hello')

    messages.append({
        "role": "tool",
        "tool_call_id": tool.id,
        "content": str(result)
    })


What would you like me to do? what is the file 'README.md' about
Message(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(function=Function(arguments='{}', name='list_files'), id='call_SzOWqFXjDrdWdTB11hs6pVZD', type='function')], function_call=None, provider_specific_fields={'refusal': None}, annotations=[])
Tool Name: list_files
Tool Arguments: {}
Result: ['.config', 'README.md', 'sample_data']
hello
Message(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(function=Function(arguments='{"file_name":"README.md"}', name='read_file'), id='call_ioEXbnScZLXtdwZ5pGQf1oUh', type='function')], function_call=None, provider_specific_fields={'refusal': None}, annotations=[])
Tool Name: read_file
Tool Arguments: {'file_name': 'README.md'}
Result: This directory includes a few sample datasets to get you started.

*   `california_housing_data*.csv` is California housing data from the 1990 US
    Census; more information is available at:
    https://

TypeError: 'NoneType' object is not subscriptable